In [1]:
import numpy as np
import matplotlib.pyplot as plt

from pyscipopt import Model
from pyscipopt import quicksum

from tqdm import tqdm, trange

np.random.seed(0)

In [2]:
def print_sudoku(_map):
    print("-------------------------")
    for i in range(9):
        line = "|"
        for j in range(9):
            line = line + " " + str(_map[i,j])
            if (j+1)%3==0:
                line = line + " |"
        print(line)
        if (i+1)%3==0:
            print("-------------------------")

def generate(percentage):
    model = Model("Sodoku")
    # Set a time limit
    model.setParam('limits/time', 60)
    
    x = {}
    for i in range(9):
        for j in range(9):
            for k in range(9):
                x[i,j,k] = model.addVar(vtype="B", name="x_"+str(i)+str(j)+str(k))
    for i in range(9):
        for j in range(9):
            model.addCons(quicksum(x[i,j,k] for k in range(9)) == 1)
    for i in range(9):
        for k in range(9):
            model.addCons(quicksum(x[i,j,k] for j in range(9)) == 1)
    for j in range(9):
        for k in range(9):
            model.addCons(quicksum(x[i,j,k] for i in range(9)) == 1)
    for k in range(9):
        for bi in range(0,9,3):
            for bj in range(0,9,3):
                model.addCons(quicksum(x[i,j,k] for i in range(bi,bi+3) for j in range(bj,bj+3)) == 1)

    for block in range(3):
        nums = np.random.choice(9,9,replace=False)
        index = 0
        for i in range(block*3, block*3+1):
            for j in range(block*3, block*3+1):
                k = nums[index]
                model.addCons(x[i,j,k] == 1)
                index+=1

    model.optimize()
    
    result = np.zeros([9,9], int)
    for i in range(9):
        for j in range(9):
            result[i,j] = [model.getVal(x[i,j,k])==1 for k in range(9)].index(True) + 1
    
    mask = np.random.random([9,9])
    index = int(np.clip(81 * percentage, 0, 80))
    mask_s = sorted(mask.reshape([-1]))
    mask = mask <= mask_s[index]
    result = result * mask
    return result

def solve(problem):
    model = Model("Sodoku")
    # Set a time limit
    model.setParam('limits/time', 60)
    
    x = {}
    for i in range(9):
        for j in range(9):
            for k in range(9):
                x[i,j,k] = model.addVar(vtype="B", name="x_"+str(i)+str(j)+str(k))
    for i in range(9):
        for j in range(9):
            model.addCons(quicksum(x[i,j,k] for k in range(9)) == 1)
    for i in range(9):
        for k in range(9):
            model.addCons(quicksum(x[i,j,k] for j in range(9)) == 1)
    for j in range(9):
        for k in range(9):
            model.addCons(quicksum(x[i,j,k] for i in range(9)) == 1)
    for k in range(9):
        for bi in range(0,9,3):
            for bj in range(0,9,3):
                model.addCons(quicksum(x[i,j,k] for i in range(bi,bi+3) for j in range(bj,bj+3)) == 1)

    for i in range(9):
        for j in range(9):
            if problem[i,j]>0:
                model.addCons(x[i,j,problem[i,j]-1] == 1)

    model.optimize()
    
    result = np.zeros([9,9], int)
    for i in range(9):
        for j in range(9):
            result[i,j] = [model.getVal(x[i,j,k])==1 for k in range(9)].index(True) + 1
    return result

In [3]:
problem = generate(percentage=0.5)
print("Problem:")
print_sudoku(problem)

print("")

print("Solution:")
solution = solve(problem)
print_sudoku(solution)

Problem:
-------------------------
| 8 0 9 | 0 6 0 | 4 0 2 |
| 0 0 0 | 8 0 2 | 9 6 0 |
| 3 6 2 | 9 4 1 | 0 8 0 |
-------------------------
| 0 8 0 | 0 9 3 | 0 0 6 |
| 1 0 0 | 0 0 0 | 3 7 4 |
| 0 5 0 | 6 0 0 | 0 2 0 |
-------------------------
| 0 0 0 | 2 5 9 | 6 1 0 |
| 9 3 0 | 0 0 6 | 2 0 7 |
| 0 0 1 | 3 7 0 | 0 0 8 |
-------------------------

Solution:
-------------------------
| 8 1 9 | 7 6 5 | 4 3 2 |
| 5 7 4 | 8 3 2 | 9 6 1 |
| 3 6 2 | 9 4 1 | 7 8 5 |
-------------------------
| 2 8 7 | 4 9 3 | 1 5 6 |
| 1 9 6 | 5 2 8 | 3 7 4 |
| 4 5 3 | 6 1 7 | 8 2 9 |
-------------------------
| 7 4 8 | 2 5 9 | 6 1 3 |
| 9 3 5 | 1 8 6 | 2 4 7 |
| 6 2 1 | 3 7 4 | 5 9 8 |
-------------------------
